In [ ]:
'''
학습 목표

-중복된 데이터를 찾아 제거할 수 있고, 결측치(missing data)를 제거하거나 채워 넣을 수 있습니다.
-데이터를 정규화시킬 수 있습니다.
-이상치(outlier)를 찾고, 이를 처리할 수 있습니다.
-범주형 데이터를 원-핫 인코딩할 수 있습니다.
-연속적인 데이터를 구간으로 나눠 범주형 데이터로 변환할 수 있습니다.
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("👽 Hello.")

In [ ]:
import os

csv_file_path = '/Users/bumchanpark/Desktop/Aiffel_Research/node/Chap2_DataPreprocess/trade.csv'
trade = pd.read_csv(csv_file_path)
trade.head()

In [ ]:
'''
2. 결측치(Missing Data)
현실에서 여러분이 다룰 데이터는 결측치를 포함하고 있는 경우가 많습니다.
물론 데이터를 수집하는 과정에서 누락되지 않도록 하는 것이 더 좋은 방법이지만,
이미 결측치가 존재한다면 이를 처리해 주어야 합니다.

결측치를 처리하는 방법은 크게 두 가지가 있습니다.

1. 결측치가 있는 데이터를 제거한다.
2. 결측치를 어떤 값으로 대체한다.
(결측치를 대체하는 방법은 다양한데, 데이터마다 특성을 반영하여 해결해야 합니다.)

우선 결측치 여부를 먼저 살펴보겠습니다.
'''

print('전체 데이터 건수:', len(trade))

In [ ]:
'''
전체 데이터 건수에서 각 컬럼별 값이 있는 데이터 수를 빼주면 컬럼별 결측치의 개수를 알 수 있습니다.
'''

print('컬럼별 결측치 개수')
len(trade) - trade.count()

In [ ]:
'''
'기타사항'을 보시면 전부 결측치라는 것을 알 수 있습니다.
이는 아무런 정보가 없는 컬럼이므로 삭제하도록 하겠습니다.
'''

trade = trade.drop('기타사항', axis=1)
trade.head()

In [ ]:
'''
기타사항 컬럼이 삭제된 것을 확인해 보았습니다. 

이제 결측치가 있는 행을 살펴보겠습니다.

DataFrame.isnull()은 데이터마다 결측치 여부를 True, False로 반환합니다.
DataFrame.any(axis=1)는 행마다 하나라도 True가 있으면 True, 그렇지 않으면 False를 반환합니다. 

두 메서드를 조합하여 결측치가 하나라도 있는 행을 찾아보겠습니다.

DataFrame에 isnull()을 적용하고, 여기도 또 any(axis=1) 메서드를 적용합니다.
이 결과, '각 행이 결측치가 하나라도 있는지' 여부를 불리언 값으로 가진 Series가 출력됩니다.
'''

trade.isnull()

In [ ]:
trade.isnull().any(axis=1)

In [ ]:
'''
trade.isnull().any(axis=1)을 다시 DataFrame에 넣어주면 값이 True인 데이터만 추출해 줍니다.
'''

trade[trade.isnull().any(axis=1)]

In [ ]:
'''
index 191 데이터는 수출금액과 무역수지 컬럼이 빠져있고,
index 196, 197, 198은 기간, 국가명을 제외하고 모두 결측치입니다.
이 경우 index 191 데이터는 삭제하기보다 특정 값으로 대체하는 것이 좋습니다.
반면 index 196, 197, 198은 제거하는 것이 바람직합니다.

우선 '수출건수', '수출금액', '수입건수', '수입금액', '무역수지' 열이
모두 결측치인 index 196, 197, 198을 삭제하겠습니다.

DataFrame의 dropna는 결측치를 삭제해 주는 메서드입니다.
subset 옵션으로 특정 컬럼들을 선택했습니다.
how 옵션으로 선택한 컬럼 전부가 결측치인 행을 삭제하겠다는 의미로 'all'을 선택합니다
('any': 하나라도 결측치인 경우). inplace 옵션으로 해당 DataFrame 내부에 바로 적용시켰습니다.
'''

trade.dropna(how='all', subset=['수출건수', '수출금액', '수입건수', '수입금액', '무역수지'], inplace=True)
print("👽 It's okay, no biggie.")

In [ ]:
# Q. 결측치가 하나라도 존재하는 데이터를 다시 확인해봅시다.
trade[trade.isnull().any(axis=1)]

In [ ]:
'''
index 191과 같이 수치형 데이터를 보완할 방법은 많습니다.

1. 특정 값을 지정해 줄 수 있습니다. 그러나 결측치가 많은 경우,
모두 같은 값으로 대체한다면 데이터의 분산이 실제보다 작아지는 문제가 생길 수 있습니다.

2. 평균, 중앙값 등으로 대체할 수 있습니다. 1번에서 특정 값으로 대체했을 때와 마찬가지로 결측치가 많은 경우
데이터의 분산이 실제보다 작아지는 문제가 발생할 수 있습니다.

3. 다른 데이터를 이용해 예측값으로 대체할 수 있습니다.
예를 들어 머신러닝 모델로 2020년 4월 미국의 예측값을 만들고, 이 값으로 결측치를 보완할 수 있습니다.

4. 시계열 특성을 가진 데이터의 경우 앞뒤 데이터를 통해 결측치를 대체할 수 있습니다.
예를 들어 기온을 측정하는 센서 데이터에서 결측치가 발생할 경우, 전후 데이터의 평균으로 보완할 수 있습니다.

index 191은 4번 방법을 통해 보완하도록 하겠습니다.

trade 데이터셋에서 국가명인 미국이며 2020년 3월과 5월 데이터셋을 출력합니다.
'''

trade[(trade['국가명']=='미국')&((trade['기간']=='2020년 03월')|(trade['기간']=='2020년 05월'))]

In [ ]:
'''
실행 결과 각 항목의 인덱스는 188, 194임을 얻었습니다.
index 191의 수출금액 컬럼값을 이전 달과 다음 달의 평균으로 채우도록 합니다.
'''

trade.loc[191, '수출금액'] = (trade.loc[188, '수출금액'] + trade.loc[194, '수출금액'] )/2
trade.loc[[191]]

In [ ]:
'''
index 191의 무역수지 컬럼은 수출금액과 수입금액의 차이를 이용하여 채우도록 하겠습니다.
'''

# Q. 무역수지 값을 채워주세요!
trade.loc[191, '무역수지'] = (trade.loc[191, '수출금액'] - trade.loc[191, '수입금액'])
trade.loc[[191]]

In [ ]:
'''
3. 중복된 데이터
데이터를 수집하는 과정에서 중복된 데이터가 생길 수 있습니다.
같은 값을 가진 데이터 없이 행(row)별로 값이 유일해야 한다면 중복된 데이터를 제거해야 합니다.

우선 중복된 데이터를 확인합니다. DataFrame.duplicated()는 데이터 중복 여부를 불리언 값으로 반환해 줍니다.
'''

trade.duplicated()

In [ ]:
trade[trade.duplicated()]

In [ ]:
trade[(trade['기간']=='2020년 03월')&(trade['국가명']=='중국')]

In [ ]:
'''
index 186, 187이 중복되어 있습니다. 

pandas에서는 DataFrame.drop_duplicates를 통해 중복된 데이터를 손쉽게 삭제할 수 있습니다.
'''

trade.drop_duplicates(inplace=True)
print("👽 It's okay, no biggie.")

In [ ]:
'''
DataFrame.drop_duplicates를 좀 더 자세히 살펴볼까요? 

다음과 같이 id와 name을 컬럼으로 갖는 df가 있다고 해봅시다.
'''

df = pd.DataFrame({'id':['001', '002', '003', '004', '002'],
                   'name':['Park Yun', 'Kim Sung', 'Park Jin', 'Lee Han', 'Kim Min']})
df

In [ ]:
'''
id가 002인 데이터가 2개 있습니다. id가 사람마다 unique 하다고 할 때, 둘 중 하나는 삭제해야 합니다. 

index가 클수록 나중에 들어온 데이터이고, 사용자가 이름을 수정했을 때 업데이트가 되지 않고 삽입이 되어 생긴 문제라고 가정합니다.
즉, id가 중복된 경우 맨 나중에 들어온 값만 남겨야 합니다.

DataFrame.drop_duplicates의 subset, keep 옵션을 통해 손쉽게 중복을 제거할 수 있습니다.
'''

# Q. 링크의 공식 문서를 참고해서,
# id가 중복된 경우 나중에 들어온 값만 남기는 코드를 작성해보세요!
# ('Kim Sung' 항목이 삭제되어야 합니다.)

df.drop_duplicates(subset=['id'], keep='last', inplace = True)
df

In [ ]:
'''
4. 이상치(Outlier)
trade 데이터에서 큰 값을 가지는 이상치가 있다고 가정해 봅시다.
이상치란 대부분 값의 범위에서 벗어나 극단적으로 크거나 작은 값을 의미합니다.
Min-Max Scaling 해보면 대부분의 값은 0에 가깝고 이상치만 1에 가까운 값을 가지게 될 것입니다.
이렇게 몇 개의 이상치 때문에 대부분 값의 차이는 의미가 거의 없어지게 됩니다.
극단적인 값이 생기는 경우를 제외하고 데이터를 고려하고 싶은 경우 이상치를 제거하고 분석합니다.

그렇다면 이상치를 어떻게 찾아내야 할까요? 현실에서 이상치를 찾는 것(anomaly detection) 자체가 큰 분야입니다.

가장 먼저 생각해 볼 수 있는 간단하고 자주 사용되는 방법은 평균과 표준편차를 이용하는 z-score 방법입니다.
평균을 빼주고 표준편차로 나눠 z-score를 계산합니다.
그리고 z score가 특정 기준을 넘어서는 데이터에 대해 이상치라고 판단합니다.
기준을 작게 하면 이상치라고 판단하는 데이터가 많아지고, 기준을 크게 하면 이상치라고 판단하는 데이터가 적어집니다.

이상치를 판단한 뒤 어떻게 해야 할까요?

1. 가장 간단한 방법으로 이상치를 삭제할 수 있습니다. 이상치를 원래 데이터에서 삭제하고, 이상치끼리 따로 분석하는 방안도 있습니다.
2. 이상치를 다른 값으로 대체할 수 있습니다. 데이터가 적으면 이상치를 삭제하기보다 다른 값으로 대체하는 것이 나을 수 있습니다.
예를 들어 최댓값, 최솟값을 설정해 데이터의 범위를 제한할 수 있습니다.
3. 혹은 결측치와 마찬가지로 다른 데이터를 활용하여 예측 모델을 만들어 예측값을 활용할 수도 있습니다.
4. 아니면 binning을 통해 수치형 데이터를 범주형으로 바꿀 수도 있습니다.

z-score method
이제 우리 데이터를 처리해 봅시다. 이상치인 데이터의 인덱스를 리턴하는 outlier라는 함수를 만들었습니다.
데이터프레임 df, 컬럼 col, 기준 z를 인풋으로 받습니다.

abs(df[col] - np.mean(df[col])) : 데이터에서 평균을 빼준 것에 절대값을 취합니다.
abs(df[col] - np.mean(df[col]))/np.std(df[col]) : 위에서 얻은 값을 표준편차로 나눠줍니다.
df[abs(df[col] - np.mean(df[col]))/np.std(df[col])>z].index : 값이 z보다 큰 데이터의 인덱스를 추출합니다.
'''

def outlier(df, col, z):
    return df[abs(df[col] - np.mean(df[col]))/np.std(df[col])>z].index
print("👽 It's okay, no biggie.")

In [ ]:
trade.loc[outlier(trade, '무역수지', 1.5)]

In [ ]:
# Q. 무역수지 값을 기준으로 z=2일 때 이상치 데이터를 출력해보세요.
trade.loc[outlier(trade, '무역수지', 2)]

In [ ]:
# Q. 무역수지 값을 기준으로 z=3일 때 이상치 데이터를 출력해보세요.
trade.loc[outlier(trade, '무역수지', 3)]

In [ ]:
'''
무역수지의 이상치를 확인하는데 기준 되는 값이 클수록 이상치가 적어지는 것을 확인할 수 있습니다. 

이제 not_outlier라는 함수를 통해 무역수지가 이상치 값이 아닌 데이터만 추출하도록 하겠습니다.
'''

# Q. not_outlier() 함수를 구현하세요.
def not_outlier(df, col, z):
    return df[abs(df[col] - np.mean(df[col]))/np.std(df[col])<=z].index

print("👽 It's okay, no biggie.")

In [ ]:
trade.loc[not_outlier(trade, '무역수지', 1.5)]

In [ ]:
'''
IQR method
하지만 이상치를 찾는 방법에는 위에 설명한 z-score 방법만 있는 것은 아닙니다.
그리고 z-score 방법은 몇 가지 뚜렷한 한계점을 가지고 있습니다.
z-score 방법의 대안으로 사분위 범위수 IQR(Interquartile range) 로 이상치를 알아내는 방법을 알아보겠습니다.

이해를 돕기 위해 아웃라이어가 포함된 임의의 데이터를 만들어보겠습니다.
'''

np.random.seed(2020)
data = np.random.randn(100)  # 평균 0, 표준편차 1의 분포에서 100개의 숫자를 샘플링한 데이터 생성
data = np.concatenate((data, np.array([8, 10, -3, -5]))) # [8, 10, -3, -5])를 데이터 뒤에 추가함
data

In [ ]:
'''
아래의 박스 플롯에서 박스를 벗어난 점들이 보이시나요?
'''

fig, ax = plt.subplots()
ax.boxplot(data)
plt.show()

In [ ]:
'''
우리는 사분위 범위수 IQR(Interquartile range)을 이용하여 이상치를 찾아낼 수 있습니다.
IQR=Q3−Q1
즉, IQR은 제 3사분위수에서 제 1사분위 값을 뺀 값으로 데이터의 중간 50%의 범위라고 생각하시면 됩니다. 
Q1−1.5×IQR보다 왼쪽에 있거나, 
Q3+1.5×IQR보다 오른쪽에 있는 경우 우리는 이상치라고 판단합니다.

IQR을 구하기 위해 우선 제1사분위수와 제 3사분위수를 구합니다.
'''

Q3, Q1 = np.percentile(data, [75 ,25])
IQR = Q3 - Q1
IQR

In [ ]:
'''
IQR과 제 1사분위수, 제 3사분위수를 이용하여 이상치를 확인할 수 있습니다.
'''

data[(Q1-1.5*IQR > data)|(Q3+1.5*IQR < data)]

In [ ]:
'''
위에서 z-score 방법과 IQR 방법에 대해서 알아보았습니다. 그럼 다음 링크의 아티클을 읽어보고 질문에 답해 봅시다.

이상치 탐지를 하는 세가지 방법
Q. z-score 방법이 가지는 뚜렷한 단점 2가지가 무엇인가요?

A.
① Robust하지 못합니다. 왜나하면 평균과 표준편차 자체가 이상치의 존재에 크게 영향을 받기 때문입니다. 
② 작은 데이터셋의 경우 z-score의 방법으로 이상치를 알아내기 어렵습니다. 특히 item이 12개 이하인 데이터셋에서는 불가능합니다.

무역수지를 기준으로 이상치를 찾는 실습을 해보도록 하겠습니다. 아래 코드에서 outlier2(df, col) 메서드를 구현해 보세요.
'''

# Q. 사분위 범위수를 이용해서 이상치를 찾는 outlier2() 함수를 구현해보세요.
def outlier2(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3-Q1
    return df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > 1.5 * IQR)]

outlier2(trade, '무역수지')

In [ ]:
'''
5. 정규화(Normalization)
rade 데이터를 보면 수입건수, 수출건수와 수입금액, 수출금액, 무역수지는 단위가 다르다는 것을 알 수 있습니다.

이처럼 컬럼마다 스케일이 크게 차이가 나는 데이터를 입력하면 머신러닝 모델 학습에 문제가 발생할 수 있습니다.
예를 들어 데이터의 범위가 0에서 1 사이인 컬럼 A과 1000에서 10000 사이인 컬럼 B가 있다고 생각해 봅시다.
이런 데이터를 클러스터링 한다고 가정해 봅시다. 데이터 간의 거리를 잴 때, 범위가 큰 컬럼 B의 값에만 영향을 크게 받을 것입니다.
다른 예시로 간단한 linear regression을 한다고 가정해 봅시다.
모델의 파라미터를 업데이트하는 과정에서 범위가 큰 컬럼 B의 파라미터만 집중적으로 업데이트하는 문제가 생길 수 있습니다.
그래서 일반적으로 컬럼 간에 범위가 크게 다를 경우 전처리 과정에서 데이터를 정규화합니다.

정규화를 하는 방법은 다양하지만, 가장 잘 알려진 표준화(Standardization)와 Min-Max Scaling을 알아보도록 하겠습니다.

1. Standardization
데이터의 평균은 0, 분산은 1로 변환합니다.
Standardization은 보통 평균이 0이고 표준편차가 1일 때 사용합니다. 그렇기에 데이터가 가우시안 분포를 따를 경우 유용합니다. 
 
2. Min-Max Scaling
데이터의 최솟값은 0, 최댓값은 1로 변환합니다.
Min-Max Scaling은 피처의 범위가 다를 때 주로 사용하며 확률 분포를 모를 때 유용합니다. 

정규화 기법이 데이터의 분포를 어떻게 바꾸는지 살펴볼까요? 우선 임의의 데이터를 생성하고, 각각의 기법으로 데이터를 정규화시켜줍니다.
'''

# 정규분포를 따라 랜덤하게 데이터 x를 생성합니다.
x = pd.DataFrame({'A': np.random.randn(100)*4+4,
                 'B': np.random.randn(100)-1})
x

In [ ]:
# 데이터 x를 Standardization 기법으로 정규화합니다. (데이터 - 데이터의 평균) / 표준편차
x_standardization = (x-x.mean())/x.std()
x_standardization

In [ ]:
# 데이터 x를 min-max scaling 기법으로 정규화합니다. (x - x최솟값) / (x최댓값 - x최솟값)
x_min_max = (x-x.min())/(x.max()-x.min())
x_min_max

In [ ]:
'''
다음 이미지는 데이터를 standardization 기법으로 정규화를 했을 때 분포가 어떻게 바뀌는지 보여줍니다.
즉, 각 컬럼의 평균은 0으로, 분산은 1로 데이터를 바꿔줍니다.
'''

fig, axs = plt.subplots(1,2, figsize=(12, 4),
                        gridspec_kw={'width_ratios': [2, 1]})

axs[0].scatter(x['A'], x['B'])
axs[0].set_xlim(-5, 15)
axs[0].set_ylim(-5, 5)
axs[0].axvline(c='grey', lw=1)
axs[0].axhline(c='grey', lw=1)
axs[0].set_title('Original Data')

axs[1].scatter(x_standardization['A'], x_standardization['B'])
axs[1].set_xlim(-5, 5)
axs[1].set_ylim(-5, 5)
axs[1].axvline(c='grey', lw=1)
axs[1].axhline(c='grey', lw=1)
axs[1].set_title('Data after standardization')

plt.show()

In [ ]:
'''
다음 이미지는 동일한 데이터를 min-max scaling 기법으로 정규화를 했을 때 분포가 어떻게 바뀌는지 보여줍니다.
즉, 각 컬럼의 최솟값은 0, 최댓값은 1로 바꿔줍니다.
'''

fig, axs = plt.subplots(1,2, figsize=(12, 4),
                        gridspec_kw={'width_ratios': [2, 1]})

axs[0].scatter(x['A'], x['B'])
axs[0].set_xlim(-5, 15)
axs[0].set_ylim(-5, 5)
axs[0].axvline(c='grey', lw=1)
axs[0].axhline(c='grey', lw=1)
axs[0].set_title('Original Data')

axs[1].scatter(x_min_max['A'], x_min_max['B'])
axs[1].set_xlim(-5, 5)
axs[1].set_ylim(-5, 5)
axs[1].axvline(c='grey', lw=1)
axs[1].axhline(c='grey', lw=1)
axs[1].set_title('Data after min-max scaling')

plt.show()

In [ ]:
'''
Standardization
우선 정규화를 시켜야 할 수치형 컬럼들을 cols 변수에 담은 후, 데이터에서 평균을 빼고, 표준편차로 나눠주도록 합니다.
'''

# trade 데이터를 standardization 기법으로 정규화합니다.
cols = ['수출건수', '수출금액', '수입건수', '수입금액', '무역수지']
trade_standardization = (trade[cols]-trade[cols].mean())/trade[cols].std()
trade_standardization.head()

In [ ]:
'''
standardization 방법으로 정규화시킨 trade_standardization을 확인해 보겠습니다.
각 컬럼의 평균들을 보면 거의 0에 가깝고, 표준편차는 1에 가까운 것을 확인하실 수 있습니다.
'''

trade_standardization.describe()

In [ ]:
'''
Min-Max Scaling
데이터에서 최솟값을 빼주고, '최댓값-최솟값'으로 나눠줍니다.
'''

# Q. trade 데이터를 min-max scaling 기법으로 정규화합니다.
for col in cols:
    min_val = trade[col].min()
    max_val = trade[col].max()
    trade[col] = (trade[col] - min_val) / (max_val - min_val)
trade.head()

In [ ]:
'''
min-max scaling 방법으로 정규화시킨 후, 각 컬럼의 최솟값(min)은 0이고, 최댓값(max)은 1임을 확인할 수 있습니다.

우리는 실제로 해당 값에 대한 분포를 정확하게 모르기 때문에 standardization보다 min-max scaling을 사용해 정규화하겠습니다.
'''

trade.describe()

In [ ]:
'''
주의!!

train 데이터와 test 데이터가 나눠져 있는 경우 train 데이터를 정규화시켰던 기준 그대로 test 데이터도 정규화시켜줘야 합니다.
'''

train = pd.DataFrame([[10, -10], [30, 10], [50, 0]])
test = pd.DataFrame([[0, 1], [10, 10]])
print("👽 It's okay, no biggie.")

In [ ]:
# Q. train 데이터와 test 데이터에 정규화를 적용해봅시다.
train_min = train.min()
train_max = train.max()

# 중요한 점은, test 데이터에 min-max scaling을 적용할 때도
# train 데이터 기준으로 수행해야 한다는 것입니다.
train_min_max = (train - train_min) / (train_max - train_min)
test_min_max = (test - train_min) / (train_max - train_min)

print("💫 It's okay, no biggie...")

In [ ]:
train_min_max

In [ ]:
test_min_max

In [ ]:
'''
scikit-learn의 StandardScaler, MinMaxScaler를 사용하는 방법도 있습니다.
'''

from sklearn.preprocessing import MinMaxScaler
train = [[10, -10], [30, 10], [50, 0]]
test = [[0, 1]]
scaler = MinMaxScaler()
print("👽 It's okay, no biggie.")

In [ ]:
%%time
scaler.fit_transform(train)

In [ ]:
scaler.transform(test)

In [ ]:
'''
6. 원-핫 인코딩(One-Hot Encoding)
이제 범주형 데이터인 국가명 컬럼을 다뤄보도록 하겠습니다. 

머신러닝이나 딥러닝 프레임워크에서 범주형을 지원하지 않는 경우 원-핫 인코딩을 해야 합니다.

원-핫 인코딩이란 카테고리별 이진 특성을 만들어 해당하는 특성만 1, 나머지는 0으로 만드는 방법입니다.
그럼, pandas로 국가명 컬럼을 원-핫 인코딩을 해보겠습니다.

pandas에서 get_dummies 함수를 통해 손쉽게 원-핫 인코딩을 할 수 있습니다.
'''

#trade 데이터의 국가명 컬럼 원본
print(trade['국가명'].head())

# get_dummies를 통해 국가명 원-핫 인코딩
country = pd.get_dummies(trade['국가명'])
country.head()

In [ ]:
'''
pd.concat 함수로 데이터프레임 trade와 country를 합쳐줍니다.
'''

trade = pd.concat([trade, country], axis=1)
trade.head()

In [ ]:
'''
이제는 필요 없어진 국가명 컬럼을 삭제해 주고 나면 trade는 우리가 원하는 데이터프레임이 됩니다.
'''

trade.drop(['국가명'], axis=1, inplace=True)
trade.head()

In [ ]:
'''
7. 구간화(Binning)
지금까지 trade 데이터를 다루면서 다양 전처리 기법을 배웠습니다. 이제 다른 전처리 기법을 배워보도록 하겠습니다.

salary에 소득 데이터가 있다고 합시다.
'''

salary = pd.Series([4300, 8370, 1750, 3830, 1840, 4220, 3020, 2290, 4740, 4600,
                    2860, 3400, 4800, 4470, 2440, 4530, 4850, 4850, 4760, 4500,
                    4640, 3000, 1880, 4880, 2240, 4750, 2750, 2810, 3100, 4290,
                    1540, 2870, 1780, 4670, 4150, 2010, 3580, 1610, 2930, 4300,
                    2740, 1680, 3490, 4350, 1680, 6420, 8740, 8980, 9080, 3990,
                    4960, 3700, 9600, 9330, 5600, 4100, 1770, 8280, 3120, 1950,
                    4210, 2020, 3820, 3170, 6330, 2570, 6940, 8610, 5060, 6370,
                    9080, 3760, 8060, 2500, 4660, 1770, 9220, 3380, 2490, 3450,
                    1960, 7210, 5810, 9450, 8910, 3470, 7350, 8410, 7520, 9610,
                    5150, 2630, 5610, 2750, 7050, 3350, 9450, 7140, 4170, 3090])
print("👽 Almost there..")

In [ ]:
'''
이 데이터를 구간별로 나누고자 합니다. 이러한 기법을 구간화(Data binning 혹은 bucketing)이라고 부릅니다.

아래 히스토그램과 같이 연속적인 데이터를 구간을 나눠 분석할 때 사용하는 방법입니다.
'''

salary.hist()

In [ ]:
'''
pandas의 cut 과 qcut을 이용해 수치형 데이터를 범주형 데이터로 변형시키도록 하겠습니다.

cut을 사용하기 위해 우선 구간을 정해줍니다.
'''

bins = [0, 2000, 4000, 6000, 8000, 10000]
print("👽 Almost there..")

In [ ]:
'''
cut 함수에 데이터와 구간을 입력하면 데이터를 구간별로 나눠줍니다.
'''

ctg = pd.cut(salary, bins=bins)
ctg

In [ ]:
'''
salary[0]는 4300으로 4000에서 6000 사이에 포함되었다는 것을 확인할 수 있습니다
'''

print('salary[0]:', salary[0])
print('salary[0]가 속한 카테고리:', ctg[0])

In [ ]:
'''
구간별로 값이 몇 개가 속해 있는지 value_counts()로 확인해 보겠습니다.
'''

ctg.value_counts().sort_index()

In [ ]:
'''
이렇게 특정 구간을 지정해 줘도 되고, 구간의 개수를 지정해 줄 수도 있습니다.
bins 옵션에 정수를 입력하면 데이터의 최솟값에서 최댓값을 균등하게 bins  개수만큼 나눠줍니다.
'''

# Q. 'bins' 옵션에 6을 입력해서 cut() 함수를 사용해보세요.
ctg = pd.cut(salary, bins = 6)
ctg

In [ ]:
# Q. 구간별로 값이 몇 개가 있는지 확인해봅시다.
ctg.value_counts().sort_index()

In [ ]:
'''
qcut은 구간을 일정하게 나누는 것이 아니라 데이터의 분포를 비슷한 크기의 그룹으로 나눠줍니다.
'''

ctg = pd.qcut(salary, q=5)
ctg

In [ ]:
print(ctg.value_counts().sort_index())
print(".\n.\n🛸 Well done!")